In [12]:
import torch
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

# 미리 작성된 코드들은 수정할 수 없으며, 이외의 코드를 작성하시면 됩니다.

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Classifier(nn.Module):
    # 모델의 코드는 여기서 작성해주세요

    def __init__(self):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(32*32*3, 512)
        self.linear2 = nn.Linear(512, 256)
        self.linear3 = nn.Linear(256, 64)
        self.linear4 = nn.Linear(64, 10)

        self.dropout = nn.Dropout(0.15) # dropout 0.15=>0.5

        self.activation = nn.ReLU()

    def forward(self, x):
        z1 = self.linear1(x)
        a1 = self.activation(z1)
        a1 = self.dropout(a1)

        z2 = self.linear2(a1)
        a2 = self.activation(z2)
        a2 = self.dropout(a2)

        z3 = self.linear3(a2)
        a3 = self.activation(z3)
        a3 = self.dropout(a3)

        z4 = self.linear4(a3)
        
        return z4


if __name__ == "__main__":
    # 학습코드는 모두 여기서 작성해주세요

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])

    train_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                 train=True,
                                                 transform=transforms.ToTensor(),
                                                 download=True)
    test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                                train=False,
                                                transform=transforms.ToTensor(),
                                                download=True)

    batch_size = 512
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle = True )
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size)
    
    model = Classifier().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00005) # optim?
    criterion = nn.CrossEntropyLoss()
    #===================================
    #            train code 
    #===================================

    epochs = 160
    lmbd = 0.1
    train_avg_costs = []
    test_avg_costs = []

    test_total_batch = len(test_dataloader)
    total_batch_num = len(train_dataloader)

    for epoch in range(epochs):
        avg_cost = 0
        model.train()
        for b_x, b_y in train_dataloader:
          b_x = b_x.view(-1, 32*32*3).to(device)
          logits = model(b_x) # forward propagation
          loss = criterion(logits, b_y.to(device)) # get cost

          reg = model.linear1.weight.pow(2.0).sum()
          reg += model.linear2.weight.pow(2.0).sum()
          reg += model.linear3.weight.pow(2.0).sum()
          reg += model.linear4.weight.pow(2.0).sum()

          loss += lmbd*reg/len(b_x)/2.

          optimizer.zero_grad()
          loss.backward() # back propagation
          optimizer.step() # update parameters

          avg_cost += loss / total_batch_num
        train_avg_costs.append(avg_cost.detach()) # point 
        print('Epoch : {} / {}, cost : {}'.format(epoch+1, epochs, avg_cost))

        test_avg_cost = 0
        model.eval()
        for b_x, b_y in test_dataloader:
          b_x = b_x.view(-1, 32*32*3).to(device)
          with torch.no_grad():
            logits = model(b_x)
            test_loss = criterion(logits, b_y.to(device))
          test_avg_cost += test_loss / test_total_batch
        
        test_avg_costs.append(test_avg_cost.detach()) # point

        # epoch = range(epochs)
        # plt.plot(epoch.train_avg_costs,'r-')
        # plt.plot(epoch.test_avg_costs,'b-')
        # plt.xlabel("Epoch")
        # plt.ylabel("Loss")
        # plt.legend(['train','test'])
        # plt.show()


    torch.save(model.state_dict(), 'model.pt')  # 학습된 모델을 저장하는 코드입니다.

Files already downloaded and verified
Files already downloaded and verified
Epoch : 1 / 160, cost : 2.255253314971924
Epoch : 2 / 160, cost : 2.082606315612793
Epoch : 3 / 160, cost : 1.9869563579559326
Epoch : 4 / 160, cost : 1.9287168979644775
Epoch : 5 / 160, cost : 1.8850489854812622
Epoch : 6 / 160, cost : 1.8459514379501343
Epoch : 7 / 160, cost : 1.8156145811080933
Epoch : 8 / 160, cost : 1.7960020303726196
Epoch : 9 / 160, cost : 1.767411231994629
Epoch : 10 / 160, cost : 1.7516019344329834
Epoch : 11 / 160, cost : 1.7313088178634644
Epoch : 12 / 160, cost : 1.7179089784622192
Epoch : 13 / 160, cost : 1.7020522356033325
Epoch : 14 / 160, cost : 1.6904759407043457
Epoch : 15 / 160, cost : 1.6746259927749634
Epoch : 16 / 160, cost : 1.6580930948257446
Epoch : 17 / 160, cost : 1.647814393043518
Epoch : 18 / 160, cost : 1.6379948854446411
Epoch : 19 / 160, cost : 1.6269488334655762
Epoch : 20 / 160, cost : 1.6143229007720947
Epoch : 21 / 160, cost : 1.6042277812957764
Epoch : 22 / 

In [13]:
# 학습된 모델의 성능을 평가하는 코드입니다.
# 아래의 코드로 평가를 진행할 예정이므로 아래의 코드가 정상 동작 해야하며, 제출전 모델의 성능을 확인하시면 됩니다.

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_dataset = torchvision.datasets.CIFAR10(root="CIFAR10/",
                                            train=False,
                                            transform=transforms.ToTensor(),
                                            download=True)


test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10000)

classifier = Classifier().to(device)
classifier.load_state_dict(torch.load('model.pt'))
classifier.eval()


for data, label in test_dataloader:
    data = data.view(-1, 32 * 32 * 3).to(device)

    with torch.no_grad():
        logits = classifier(data)

        pred = torch.argmax(logits, dim=1)

        total = len(label)
        correct = torch.eq(pred, label.to(device)).sum()

        print("Accuracy on test set : {:.4f}%".format(100 * correct / total))

Files already downloaded and verified
Accuracy on test set : 56.1200%
